## Best Tree Selection and RF Distance Analysis

This script processes multiple datasets (DNA/AA × easy/medium/hard) to identify the best-scoring phylogenetic tree for each parameter setting (epsilon). For every dataset, all runs are scanned and the tree with the highest final log-likelihood is chosen. The tree from the *smallest epsilon* serves as the reference. Each best tree is compared to the reference tree using RF distance (absolute and normalized). A structured table with all distances, parameters, and file paths is saved as **`Best_Tree_RF_Distances.csv`**. This allows consistent comparison of phylogenetic tree stability across parameter settings.

In [2]:
from ete3 import Tree
import os
import glob
import pandas as pd

BASE_DIR = "."

# Desired order
datasets = ["dna_easy", "dna_medium", "dna_hard",
             "aa_easy", "aa_medium", "aa_hard"]

def is_float(x):
    try:
        float(x)
        return True
    except:
        return False

rows = []

for dataset in datasets:
    dataset_path = os.path.join(BASE_DIR, dataset)
    if not os.path.isdir(dataset_path):
        print(f"⚠️ Dataset folder missing: {dataset}")
        continue

    print(f"\n📂 DATASET: {dataset}")

    # Step 1: Find the best tree for each parameter
    best_trees = {}

    # Extract only parameter folders (anything interpretable as a float)
    params = [p for p in os.listdir(dataset_path)
              if os.path.isdir(os.path.join(dataset_path, p)) and is_float(p)]

    # Sort numerically
    params = sorted(params, key=lambda x: float(x))

    for param in params:
        param_path = os.path.join(dataset_path, param)

        run_candidates = []

        for runfolder in os.listdir(param_path):
            run_path = os.path.join(param_path, runfolder)
            if not os.path.isdir(run_path):
                continue

            # Look for the final tree
            tree_files = glob.glob(os.path.join(run_path, "*_tree.newick"))
            if not tree_files:
                continue

            # Determine the Likelihood
            logl_files = glob.glob(os.path.join(run_path, "*_logl.out"))
            if logl_files:
                try:
                    with open(logl_files[0], "r") as f:
                        # Reads the last line where the final LL is located
                        ll = float(f.readlines()[-1].strip()) 
                except:
                    ll = None
            else:
                ll = None

            run_candidates.append((ll, tree_files[0]))

        if len(run_candidates) == 0:
            continue

        # Sort: best likelihood first
        run_candidates.sort(key=lambda x: (x[0] is None, -(x[0] or -9e99))) 

        best_tree_file = run_candidates[0][1]
        best_trees[param] = best_tree_file

    if len(best_trees) == 0:
        continue

    # Step 2: Reference Tree = most precise parameter
    precise_param = sorted(best_trees.keys(), key=lambda x: float(x))[0]
    ref_tree_file = best_trees[precise_param]
    ref_tree = Tree(ref_tree_file)

    print(f"🏆 Reference Parameter (Epsilon): {precise_param}")

    # Step 3: Calculate RF per parameter
    for param, tree_file in best_trees.items():
        t = Tree(tree_file)
        # Calculation of Robinson-Foulds distance
        rf, max_rf, *_ = ref_tree.robinson_foulds(t) 
        norm_rf = rf / max_rf if max_rf > 0 else 0

        comment = "Reference (Highest Precision)" if param == precise_param else "Comparison"

        rows.append({
            "Dataset": dataset,
            "Epsilon": param,
            "Tree_File_Path": tree_file,
            "RF_Distance_Absolute": rf, 
            "RF_Distance_Max_Possible": max_rf, 
            "RF_Distance_Normalized": norm_rf, 
            "Note": comment
        })

# Final Table
df = pd.DataFrame(rows)

# Maintain dataset order
df["Dataset"] = pd.Categorical(df["Dataset"], categories=datasets, ordered=True)
df = df.sort_values(["Dataset", "Epsilon"]).reset_index(drop=True)

## --- NEW SECTION: Saving the CSV file ---
# Define the desired output filename
output_filename = "Best_Tree_RF_Distances.csv"

# Save the DataFrame as CSV
df.to_csv(output_filename, index=False)

print(f"\n✅ Successfully saved as: {output_filename}")
print("---")

df

⚠️ Dataset folder missing: dna_easy
⚠️ Dataset folder missing: dna_medium
⚠️ Dataset folder missing: dna_hard
⚠️ Dataset folder missing: aa_easy
⚠️ Dataset folder missing: aa_medium
⚠️ Dataset folder missing: aa_hard


KeyError: 'Dataset'